In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/home/nico/Projets/kaggle_competitions/playground-series-s4e8')
from variables import * 


In [2]:
train = pd.read_parquet('/home/nico/Projets/kaggle_competitions/playground-series-s4e8/data/train.parquet')
train[TARGET_COL] = train[TARGET_COL].map(dict_label_target)
train.head()

/home/nico/Projets/kaggle_competitions/venv-kaggle/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/home/nico/Projets/kaggle_competitions/venv-kaggle/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,1,8.796875,f,s,u,f,a,c,w,...,None,None,w,None,None,f,f,None,d,a
1,1,0,4.511719,x,h,o,f,a,c,n,...,None,y,o,None,None,t,z,None,d,w
2,2,1,6.941406,f,s,b,f,x,c,w,...,None,s,n,None,None,f,f,None,l,w
3,3,1,3.880859,f,y,g,f,s,None,g,...,None,None,w,None,None,f,f,None,d,u
4,4,1,5.851562,x,l,w,f,d,None,w,...,None,None,w,None,None,f,f,None,g,a


## FEATURES ENGINEERING WITH TARGET ENCODING

In [3]:
def get_dict_target_encoding(df,column,target) :

    dict_target = df[[column,target]].groupby([column],dropna=False).mean()[target].to_dict()
    
    return(dict_target)

In [4]:
dict_target_encoding = {}
for cat_col in CAT_COLS :
    train[cat_col] = train[cat_col].fillna('NA') # to simplify target encoding
    dict_target_encoding[cat_col] = get_dict_target_encoding(train,cat_col,TARGET_COL)

In [5]:
for col in CAT_COLS : 
    train[col] = train[col].map(dict_target_encoding[col])
train.head()

/home/nico/Projets/kaggle_competitions/venv-kaggle/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/home/nico/Projets/kaggle_competitions/venv-kaggle/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,1,8.796875,0.488056,0.630657,0.460682,0.444237,0.353656,0.427554,0.571560,...,0.460073,0.487341,0.582286,0.458707,0.453328,0.466746,0.453262,0.470945,0.471161,0.428414
1,1,0,4.511719,0.489834,0.529871,0.282739,0.444237,0.353656,0.427554,0.285121,...,0.460073,0.299241,0.403340,0.458707,0.453328,0.408896,0.003841,0.470945,0.471161,0.653991
2,2,1,6.941406,0.488056,0.630657,0.873958,0.444237,0.461549,0.427554,0.571560,...,0.460073,0.604493,0.389072,0.458707,0.453328,0.466746,0.453262,0.470945,0.605078,0.653991
3,3,1,3.880859,0.488056,0.538017,0.588080,0.444237,0.410843,0.429159,0.519004,...,0.460073,0.487341,0.582286,0.458707,0.453328,0.466746,0.453262,0.470945,0.471161,0.417539
4,4,1,5.851562,0.489834,0.571190,0.502941,0.444237,0.428623,0.429159,0.571560,...,0.460073,0.487341,0.582286,0.458707,0.453328,0.466746,0.453262,0.470945,0.325398,0.428414


## XGBOOST 

In [6]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.metrics import matthews_corrcoef

In [7]:
X_train,X_test = train_test_split(train,train_size=0.25,random_state=42)

In [8]:
FEATURES_COL = CONT_COLS + CAT_COLS

In [9]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.5, 0.7, 1]
}

param_grid = {
    'max_depth': [ 5],
    'learning_rate': [ 0.01],
    'subsample': [ 0.7]
}

In [10]:
xgb_model = xgb.XGBClassifier(random_state = 42)
grid_search = GridSearchCV(xgb_model, param_grid, cv=2, scoring='accuracy',verbose=3)

In [11]:
grid_search.fit(X_train[FEATURES_COL], X_train[TARGET_COL])

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END learning_rate=0.01, max_depth=5, subsample=0.7;, score=0.914 total time=   7.3s
[CV 2/2] END learning_rate=0.01, max_depth=5, subsample=0.7;, score=0.912 total time=   6.7s


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'learning_rate': [0.01], 'max_depth': [5],
                         'subsample': [0.7]},
             scoring='accuracy', verbose=3)

In [12]:
# Print the best set of hyperparameters and the corresponding score
print("Best set of hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)
bst_estimator = grid_search.best_estimator_

Best set of hyperparameters:  {'learning_rate': 0.01, 'max_depth': 5, 'subsample': 0.7}
Best score:  0.9132342448244177


In [13]:
preds = bst_estimator.predict(X_test[FEATURES_COL])
score = matthews_corrcoef(X_test[TARGET_COL], preds)
print('MCC:', score)

MCC: 0.8234376545272006


## Submission

In [14]:
test = pd.read_parquet('/home/nico/Projets/kaggle_competitions/playground-series-s4e8/data/test.parquet')

In [15]:
for cat_col in CAT_COLS :
    test[cat_col] = test[cat_col].fillna('NA')
    test[cat_col] = test[cat_col].map(dict_target_encoding[cat_col]).fillna(np.mean(list(dict_target_encoding[cat_col].values())) )

In [ ]:
for cont_col in CONT_COLS :
    test[cont_col] = test[cont_col].fillna(test[cont_col].dropna().mean())

In [27]:
test['class'] = bst_estimator.predict(test[FEATURES_COL])
test['class'] = test['class'].map(dict_target_label)

In [29]:
test[['id','class']].to_csv('submission.csv',index=False)